In [1]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss-gpu]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-96_jk15p/farm-haystack_3357647688f743c7825aa284279aff53
  Resolved https://github.com/deepset-ai/haystack.git to commit 75ef9596780a18b4c5ec83777aeede1aef53be89
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━

DEPRECATION: git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss-gpu] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-96_jk15p/farm-haystack_3357647688f743c7825aa284279aff53


In [2]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.WARNING)


In [3]:
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(embedding_dim=768, faiss_index_factory_str="Flat")

In [4]:
import os
print(os.getcwd())
os.chdir('./drive/MyDrive/pale-companion-files/')

/content


In [5]:
# Import some files
import pickle
with open('./chapter_fmt_list.pkl','rb') as f:
    chapters = pickle.load(f)

# Temporary
# chapters = chapters[:100]

In [6]:
from haystack import Document
chapter_documents = [Document.from_dict(d) for d in chapters]
len(chapter_documents)

307

In [7]:
from haystack.nodes import PreProcessor

word_preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
    split_overlap=20,
    progress_bar=True, 
    add_page_number=True
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
docs_word = word_preprocessor.process(chapter_documents)
len(docs_word)

Preprocessing:   0%|          | 0/307 [00:00<?, ?docs/s]

53227

In [9]:
# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs_word)

Writing Documents:   0%|          | 0/53227 [00:00<?, ?it/s]

In [10]:
from haystack.nodes import DensePassageRetriever

retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-multiset-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-multiset-base",
    embed_title=False
    # could sub the models for the facebook ones with larget embedding dims for testing
)

document_store.update_embeddings(retriever)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


Updating Embedding:   0%|          | 0/53227 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/3232 [00:00<?, ? Docs/s]

In [11]:
document_store.save("pale_lfqa_v1_fb_multiset.faiss")

In [12]:
from haystack.utils import print_documents
from haystack.pipelines import DocumentSearchPipeline

p_retrieval = DocumentSearchPipeline(retriever)
res = p_retrieval.run(query="Tell me something about Avery?", params={"Retriever": {"top_k": 10}})
print_documents(res, max_text_len=512)



Query: Tell me something about Avery?

{   'content': '“It took too long.  We needed someone to go, hey, be friends, '
               'work together.  We weren’t paying enough attention.”  Nora '
               'nodded.  It was interesting to look at this interaction as an '
               'insight into what Nora was thinking and feeling when it came '
               'to Avery.  Given a first chance to ask something relevant, '
               'what did she ask about?  This, apparently, and-  “She’s '
               'mysterious.”  Lucy and Verona nodded.  That was trickier.  '
               '“She’s been getting a lot of phone calls.  And emails.  '
               'Texts.  ',
    'name': None}

{   'content': '“What do you think this is, Avery Kelly?”  The use of her name '
               'made her shiver.  Snowdrop had done the same thing.  “What do '
               'you mean?”  “What story is this?  All things Other have some '
               'roots in the annals of man,” the baby op

In [13]:
from haystack.nodes import Seq2SeqGenerator


generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")


In [14]:
from haystack.pipelines import GenerativeQAPipeline

pipe = GenerativeQAPipeline(generator, retriever)


In [15]:
pipe.run(query="Tell me something about Avery?", params={"Retriever": {"top_k": 10}})


{'query': 'Tell me something about Avery?',
 'answers': [<Answer {'answer': "I don't know much about Avery, but I do know a lot about her dad. He's a lawyer who specializes in child abuse cases, and he's been involved in a lot of high profile cases in the last few years. He was involved in the murder of his ex-wife, who was pregnant at the time of the murder, and the death of his son, who is also a lawyer. He also has been involved with a number of other high profile murder cases, including the murders of his brother and sister, as well as the deaths of his wife and son. I don't have any information on Avery's relationship with her father, so I can't tell you much about her.", 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['295a31ed5b5f5155b02e1859cf4fbabb', 'fe55e618a3de0891566d285b888bfdea', 'd3e952a768a0dc382b345996be52dde4', '396a3100355c944717c1727ef062c34a', '32192a17c55b0899a757c8bf9fd85d0a', '12bb7